In [1]:
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
train_joke_df = pd.read_csv(r'..\data\recsys-in-practice\train_joke_df.csv')

joke_df = pd.read_csv(r'..\clip\joke_clip_vectors.csv')

In [3]:
train_joke_df = train_joke_df.merge(joke_df, left_on='JID', right_index=True)
train_joke_df

,UID,JID,Rating,joke_feature_1,joke_feature_2,joke_feature_3,joke_feature_4,joke_feature_5,joke_feature_6,joke_feature_7,...,joke_feature_1015,joke_feature_1016,joke_feature_1017,joke_feature_1018,joke_feature_1019,joke_feature_1020,joke_feature_1021,joke_feature_1022,joke_feature_1023,joke_feature_1024
0,18029,6,-1.26,-0.032410,0.113159,-0.112366,-0.001542,-0.040222,-0.112244,0.104187,...,-0.027679,-0.119324,0.079102,0.024918,-0.002968,0.009186,0.055908,0.091553,-0.003986,-0.062103
17,5621,6,3.16,-0.032410,0.113159,-0.112366,-0.001542,-0.040222,-0.112244,0.104187,...,-0.027679,-0.119324,0.079102,0.024918,-0.002968,0.009186,0.055908,0.091553,-0.003986,-0.062103
101,17251,6,-3.74,-0.032410,0.113159,-0.112366,-0.001542,-0.040222,-0.112244,0.104187,...,-0.027679,-0.119324,0.079102,0.024918,-0.002968,0.009186,0.055908,0.091553,-0.003986,-0.062103
142,2786,6,6.36,-0.032410,0.113159,-0.112366,-0.001542,-0.040222,-0.112244,0.104187,...,-0.027679,-0.119324,0.079102,0.024918,-0.002968,0.009186,0.055908,0.091553,-0.003986,-0.062103
273,14079,6,2.09,-0.032410,0.113159,-0.112366,-0.001542,-0.040222,-0.112244,0.104187,...,-0.027679,-0.119324,0.079102,0.024918,-0.002968,0.009186,0.055908,0.091553,-0.003986,-0.062103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1447099,22356,74,1.89,-0.052185,0.034882,-0.084961,-0.002357,-0.036743,0.070374,0.057220,...,-0.037323,-0.092529,-0.071167,0.021759,-0.001674,-0.011864,0.032074,-0.010201,0.085083,-0.132568
1447290,4234,74,7.04,-0.052185,0.034882,-0.084961,-0.002357,-0.036743,0.070374,0.057220,...,-0.037323,-0.092529,-0.071167,0.021759,-0.001674,-0.011864,0.032074,-0.010201,0.085083,-0.132568
1447538,16115,74,0.34,-0.052185,0.034882,-0.084961,-0.002357,-0.036743,0.070374,0.057220,...,-0.037323,-0.092529,-0.071167,0.021759,-0.001674,-0.011864,0.032074,-0.010201,0.085083,-0.132568
1447727,11579,74,6.17,-0.052185,0.034882,-0.084961,-0.002357,-0.036743,0.070374,0.057220,...,-0.037323,-0.092529,-0.071167,0.021759,-0.001674,-0.011864,0.032074,-0.010201,0.085083,-0.132568


In [4]:
train_df, valid_df = train_test_split(train_joke_df, test_size=0.001, random_state=42)

In [5]:
# сделаем сортировку и перепишем index
train_df = train_df.sort_values(by=['UID', 'JID'])
train_df = train_df.reset_index(drop=True)

valid_df = valid_df.sort_values(by=['UID', 'JID'])
valid_df = valid_df.reset_index(drop=True)

In [6]:
from catboost import CatBoostRanker, Pool, MetricVisualizer, CatBoostRegressor
from copy import deepcopy

In [7]:
cat_features = ['UID', 'JID']

In [8]:
train_pool = Pool(train_df.drop(columns='Rating'), label=train_df['Rating'], group_id=train_df['UID'],cat_features=cat_features)
valid_pool = Pool(valid_df.drop(columns='Rating'), label=valid_df['Rating'], group_id=valid_df['UID'],cat_features=cat_features)

In [9]:
default_parameters = {
    'iterations': 1500,
    'custom_metric': 'RMSE',
    'random_seed': 0,
    'train_dir':'RMSE',
    'objective':'RMSE',
    'loss_function':'RMSE',
    'eval_metric':'RMSE'
}


In [10]:
model = CatBoostRegressor(**default_parameters)
model.fit(train_pool, eval_set=valid_pool, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.124536
0:	learn: 5.0941787	test: 4.9136755	best: 4.9136755 (0)	total: 2s	remaining: 50m
1:	learn: 4.9822973	test: 4.7934364	best: 4.7934364 (1)	total: 3.58s	remaining: 44m 40s
2:	learn: 4.8928061	test: 4.6961344	best: 4.6961344 (2)	total: 5.33s	remaining: 44m 22s
3:	learn: 4.8199752	test: 4.6186282	best: 4.6186282 (3)	total: 6.87s	remaining: 42m 50s
4:	learn: 4.7613759	test: 4.5580479	best: 4.5580479 (4)	total: 7.8s	remaining: 38m 53s
5:	learn: 4.7154361	test: 4.5069280	best: 4.5069280 (5)	total: 9.28s	remaining: 38m 31s
6:	learn: 4.6774974	test: 4.4683221	best: 4.4683221 (6)	total: 10.7s	remaining: 38m 10s
7:	learn: 4.6479580	test: 4.4404006	best: 4.4404006 (7)	total: 12s	remaining: 37m 19s
8:	learn: 4.6231170	test: 4.4131690	best: 4.4131690 (8)	total: 13.4s	remaining: 37m 7s
9:	learn: 4.6039697	test: 4.3937317	best: 4.3937317 (9)	total: 14.2s	remaining: 35m 17s
10:	learn: 4.5882092	test: 4.3783984	best: 4.3783984 (10)	total: 15.7s	remaining: 35m 27s
11:	learn: 

91:	learn: 4.4965556	test: 4.3080408	best: 4.3077068 (85)	total: 2m 10s	remaining: 33m 24s
92:	learn: 4.4963754	test: 4.3080196	best: 4.3077068 (85)	total: 2m 12s	remaining: 33m 30s
93:	learn: 4.4958616	test: 4.3082050	best: 4.3077068 (85)	total: 2m 15s	remaining: 33m 41s
94:	learn: 4.4957941	test: 4.3078153	best: 4.3077068 (85)	total: 2m 16s	remaining: 33m 44s
95:	learn: 4.4956785	test: 4.3076968	best: 4.3076968 (95)	total: 2m 18s	remaining: 33m 44s
96:	learn: 4.4954802	test: 4.3075875	best: 4.3075875 (96)	total: 2m 20s	remaining: 33m 48s
97:	learn: 4.4954134	test: 4.3077463	best: 4.3075875 (96)	total: 2m 22s	remaining: 33m 51s
98:	learn: 4.4953017	test: 4.3077743	best: 4.3075875 (96)	total: 2m 23s	remaining: 33m 54s
99:	learn: 4.4951801	test: 4.3076587	best: 4.3075875 (96)	total: 2m 25s	remaining: 33m 50s
100:	learn: 4.4950213	test: 4.3081383	best: 4.3075875 (96)	total: 2m 27s	remaining: 33m 58s
101:	learn: 4.4947896	test: 4.3076352	best: 4.3075875 (96)	total: 2m 29s	remaining: 34m 2

180:	learn: 4.4844717	test: 4.3021406	best: 4.3011246 (160)	total: 4m 25s	remaining: 32m 17s
181:	learn: 4.4843334	test: 4.3018897	best: 4.3011246 (160)	total: 4m 27s	remaining: 32m 18s
182:	learn: 4.4842347	test: 4.3018489	best: 4.3011246 (160)	total: 4m 28s	remaining: 32m 15s
183:	learn: 4.4841949	test: 4.3018156	best: 4.3011246 (160)	total: 4m 29s	remaining: 32m 9s
184:	learn: 4.4841355	test: 4.3017788	best: 4.3011246 (160)	total: 4m 31s	remaining: 32m 8s
185:	learn: 4.4840841	test: 4.3017291	best: 4.3011246 (160)	total: 4m 32s	remaining: 32m 2s
186:	learn: 4.4838210	test: 4.3014240	best: 4.3011246 (160)	total: 4m 33s	remaining: 31m 59s
187:	learn: 4.4837071	test: 4.3014237	best: 4.3011246 (160)	total: 4m 34s	remaining: 31m 56s
188:	learn: 4.4836710	test: 4.3014496	best: 4.3011246 (160)	total: 4m 35s	remaining: 31m 54s
189:	learn: 4.4835760	test: 4.3011821	best: 4.3011246 (160)	total: 4m 37s	remaining: 31m 55s
190:	learn: 4.4835360	test: 4.3010312	best: 4.3010312 (190)	total: 4m 39s

269:	learn: 4.4740058	test: 4.2909845	best: 4.2909845 (269)	total: 6m 37s	remaining: 30m 9s
270:	learn: 4.4739475	test: 4.2909477	best: 4.2909477 (270)	total: 6m 39s	remaining: 30m 10s
271:	learn: 4.4738227	test: 4.2909925	best: 4.2909477 (270)	total: 6m 40s	remaining: 30m 9s
272:	learn: 4.4737636	test: 4.2907629	best: 4.2907629 (272)	total: 6m 41s	remaining: 30m 5s
273:	learn: 4.4737270	test: 4.2905860	best: 4.2905860 (273)	total: 6m 43s	remaining: 30m 4s
274:	learn: 4.4736768	test: 4.2904240	best: 4.2904240 (274)	total: 6m 44s	remaining: 30m 2s
275:	learn: 4.4736024	test: 4.2902447	best: 4.2902447 (275)	total: 6m 45s	remaining: 29m 58s
276:	learn: 4.4735004	test: 4.2898986	best: 4.2898986 (276)	total: 6m 47s	remaining: 29m 57s
277:	learn: 4.4734274	test: 4.2897644	best: 4.2897644 (277)	total: 6m 49s	remaining: 29m 58s
278:	learn: 4.4733957	test: 4.2896884	best: 4.2896884 (278)	total: 6m 50s	remaining: 29m 55s
279:	learn: 4.4733514	test: 4.2896710	best: 4.2896710 (279)	total: 6m 52s	r

...

1236:	learn: 4.4124567	test: 4.2305836	best: 4.2305836 (1236)	total: 31m 7s	remaining: 6m 36s
1237:	learn: 4.4124204	test: 4.2306081	best: 4.2305836 (1236)	total: 31m 8s	remaining: 6m 35s
1238:	learn: 4.4123821	test: 4.2306219	best: 4.2305836 (1236)	total: 31m 10s	remaining: 6m 34s
1239:	learn: 4.4123541	test: 4.2306708	best: 4.2305836 (1236)	total: 31m 11s	remaining: 6m 32s
1240:	learn: 4.4123098	test: 4.2306007	best: 4.2305836 (1236)	total: 31m 14s	remaining: 6m 31s
1241:	learn: 4.4122178	test: 4.2304487	best: 4.2304487 (1241)	total: 31m 16s	remaining: 6m 29s
1242:	learn: 4.4121625	test: 4.2303995	best: 4.2303995 (1242)	total: 31m 17s	remaining: 6m 28s
1243:	learn: 4.4121379	test: 4.2304571	best: 4.2303995 (1242)	total: 31m 18s	remaining: 6m 26s
1244:	learn: 4.4121138	test: 4.2303661	best: 4.2303661 (1244)	total: 31m 19s	remaining: 6m 24s
1245:	learn: 4.4120805	test: 4.2302826	best: 4.2302826 (1245)	total: 31m 21s	remaining: 6m 23s
1246:	learn: 4.4119673	test: 4.2303696	best: 4.23028

1323:	learn: 4.4084243	test: 4.2287959	best: 4.2284066 (1321)	total: 33m 23s	remaining: 4m 26s
1324:	learn: 4.4083976	test: 4.2287421	best: 4.2284066 (1321)	total: 33m 25s	remaining: 4m 24s
1325:	learn: 4.4083873	test: 4.2287402	best: 4.2284066 (1321)	total: 33m 26s	remaining: 4m 23s
1326:	learn: 4.4082020	test: 4.2288031	best: 4.2284066 (1321)	total: 33m 28s	remaining: 4m 21s
1327:	learn: 4.4081594	test: 4.2288325	best: 4.2284066 (1321)	total: 33m 30s	remaining: 4m 20s
1328:	learn: 4.4081273	test: 4.2290516	best: 4.2284066 (1321)	total: 33m 32s	remaining: 4m 18s
1329:	learn: 4.4080727	test: 4.2289465	best: 4.2284066 (1321)	total: 33m 33s	remaining: 4m 17s
1330:	learn: 4.4080020	test: 4.2287937	best: 4.2284066 (1321)	total: 33m 34s	remaining: 4m 15s
1331:	learn: 4.4079827	test: 4.2287420	best: 4.2284066 (1321)	total: 33m 35s	remaining: 4m 14s
1332:	learn: 4.4079328	test: 4.2287441	best: 4.2284066 (1321)	total: 33m 37s	remaining: 4m 12s
1333:	learn: 4.4079074	test: 4.2286644	best: 4.228

1410:	learn: 4.4051128	test: 4.2250030	best: 4.2250030 (1410)	total: 35m 35s	remaining: 2m 14s
1411:	learn: 4.4050528	test: 4.2247859	best: 4.2247859 (1411)	total: 35m 36s	remaining: 2m 13s
1412:	learn: 4.4049670	test: 4.2247637	best: 4.2247637 (1412)	total: 35m 38s	remaining: 2m 11s
1413:	learn: 4.4049558	test: 4.2248157	best: 4.2247637 (1412)	total: 35m 38s	remaining: 2m 10s
1414:	learn: 4.4049508	test: 4.2248248	best: 4.2247637 (1412)	total: 35m 40s	remaining: 2m 8s
1415:	learn: 4.4049276	test: 4.2249339	best: 4.2247637 (1412)	total: 35m 41s	remaining: 2m 7s
1416:	learn: 4.4049092	test: 4.2248998	best: 4.2247637 (1412)	total: 35m 43s	remaining: 2m 5s
1417:	learn: 4.4048905	test: 4.2247009	best: 4.2247009 (1417)	total: 35m 44s	remaining: 2m 4s
1418:	learn: 4.4047336	test: 4.2245787	best: 4.2245787 (1418)	total: 35m 47s	remaining: 2m 2s
1419:	learn: 4.4047030	test: 4.2245235	best: 4.2245235 (1419)	total: 35m 49s	remaining: 2m 1s
1420:	learn: 4.4046437	test: 4.2244273	best: 4.2244273 (

1497:	learn: 4.4012977	test: 4.2228045	best: 4.2225808 (1488)	total: 37m 44s	remaining: 3.02s
1498:	learn: 4.4011840	test: 4.2224800	best: 4.2224800 (1498)	total: 37m 46s	remaining: 1.51s
1499:	learn: 4.4011476	test: 4.2224233	best: 4.2224233 (1499)	total: 37m 47s	remaining: 0us

bestTest = 4.222423294
bestIteration = 1499



CatBoostError: bad allocation

In [ ]:
predict = model.predict(valid_pool)
print(mean_squared_error(valid_df['Rating'].values, predict, squared=False))

predict = model.predict(train_pool)
print(mean_squared_error(train_df['Rating'].values, predict, squared=False))


In [ ]:
model_name = "catboost_clip"

In [ ]:
model.save_model(model_name)

from_file = CatBoostRegressor()

from_file.load_model(model_name)


In [ ]:
predict = from_file.predict(valid_pool)
print(mean_squared_error(valid_df['Rating'].values, predict, squared=False))

predict = from_file.predict(train_pool)
print(mean_squared_error(train_df['Rating'].values, predict, squared=False))


In [ ]:
assert False

In [ ]:
import gc

In [ ]:
gc.collect()

In [ ]:
del train_pool

In [ ]:
test_joke_df_nofactrating = pd.read_csv(r'..\data\recsys-in-practice\test_joke_df_nofactrating.csv', index_col=0)

test_joke_df_nofactrating = test_joke_df_nofactrating.merge(joke_df, how='left', left_on='JID', right_index=True)
test_joke_df_nofactrating
test_joke_df_nofactrating = test_joke_df_nofactrating.sort_values(by=['UID', 'JID'])
test_pool = Pool(test_joke_df_nofactrating, group_id=test_joke_df_nofactrating['UID'], cat_features=cat_features)

In [ ]:
predict = model.predict(test_pool)

test_joke_df_nofactrating['Rating'] = predict

display(test_joke_df_nofactrating['Rating'].to_frame().head(5))
test_joke_df_nofactrating['Rating'].to_frame().to_csv('catboost_clip.csv')

In [ ]:
test_joke_df_nofactrating